<a href="https://colab.research.google.com/github/heber-augusto/sus-kpis-analysis/blob/main/notebooks/validacao_estadiamento_por_ano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração do ambiente e carregamento do catálogo

**Lembre-se de baixar o arquivo json que garante acesso ao google storage**
 - monitor-rosa-escrita.json: acesso para criação de tabelas;
 - monitor-rosa-leitura.json: acesso para analises e consultas.

**Se o seu usuário possui acesso ao drive compartilhado, remova os comentários a seguir**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Escolha do ambiente
São disponibilizadas duas opções de execução (dependendo do arquivo de credencial utilizado)
 - gcp-leitura.json: disponível para analises e usuários que não possuem permissão de escrita;
 - gcp-escrita.json: disponível para atualização de bases (testes de desenvolvimento ou ambiente de produção).

In [2]:
import os
if os.path.isfile('/content/monitor-rosa-leitura.json'):
    datalake_mode = 'leitura'
    %env SERVICE_ACCOUNT_USER=acesso-leitura@monitor-rosa.iam.gserviceaccount.com
    %env SERVICE_ACCOUNT_JSON=/content/monitor-rosa-leitura.json
elif os.path.isfile('/content/monitor-rosa-escrita.json'):
    datalake_mode = 'escrita'
    %env SERVICE_ACCOUNT_USER=acesso-escrita@monitor-rosa.iam.gserviceaccount.com
    %env SERVICE_ACCOUNT_JSON=/content/monitor-rosa-escrita.json
else:
    assert(os.path.isdir('/content/drive/Shareddrives/monitor-rosa-gold') == True)
    datalake_mode = 'shared_drive'
    %env SERVICE_ACCOUNT_USER=''
    %env SERVICE_ACCOUNT_JSON=''
datalake_mode

env: SERVICE_ACCOUNT_USER=''
env: SERVICE_ACCOUNT_JSON=''


'shared_drive'

In [3]:
!rm -r sus-kpis-analysis
!git clone https://github.com/heber-augusto/sus-kpis-analysis.git

rm: cannot remove 'sus-kpis-analysis': No such file or directory
Cloning into 'sus-kpis-analysis'...
remote: Enumerating objects: 1934, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 1934 (delta 124), reused 12 (delta 12), pack-reused 1740 (from 3)
Receiving objects: 100% (1934/1934), 4.85 MiB | 13.14 MiB/s, done.
Resolving deltas: 100% (924/924), done.


## Instalação de libs Python, inicialização de variáveis de ambiente e configuração/instalação do Spark

In [4]:
!pip install -r /content/sus-kpis-analysis/sia/etls/requirements.txt

%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/content/spark-3.4.4-bin-hadoop3
%env SPARK_VERSION=3.4.4

!source /content/sus-kpis-analysis/sia/etls/bin/setup_spark_env.sh '/content/'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 70.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.4-py2.py3-none-any.whl size=311905464 sha256=f9667f7441fbc2199eb387aff62e09ad4cad9e10290344ae1b539d3c1afb636d
  Stored in directory: /root/.cache/pip/wheels/6b/0a/a1/2b8f5f192c7df9fdceb8e5a62873d64e46b101f980519bcf55
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.5
    Uninstalling pyspark-3.5.5:
      Successfully uninstalled pyspark-3.5.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-spark-connect 0.5.2 requires pyspark>=3.5, but you have pyspark 3.4.4 which is incompati

In [5]:
if datalake_mode != 'shared_drive':
    %env XDG_CONFIG_HOME=/content/datalake
    !source /content/sus-kpis-analysis/sia/etls/bin/install-google-drive-ocamlfuse.sh
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0ABIY-a4qrdY9Uk9PVA' 'monitor-rosa-bronze' $SERVICE_ACCOUNT_JSON '/content'
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0ALl0owLNr53oUk9PVA' 'monitor-rosa-silver' $SERVICE_ACCOUNT_JSON '/content'
    !source /content/sus-kpis-analysis/sia/etls/bin/mount_google_drive_v2.sh '/content/datalake' $SERVICE_ACCOUNT_USER '0AMHp9pBeLvZiUk9PVA' 'monitor-rosa-gold' $SERVICE_ACCOUNT_JSON '/content'


## Inicializa variáveis de acesso ao delta lake criado no google storage

- Local do arquivo de credencial do storage, diretorio do warehouse e path do spark:

> O arquivo json_file_name deve ser enviado para o ambiente e deve ser utilizado um com as devidas permissões (em caso de escrita)

> O caminho do warehouse pode ser alterado em caso de testes de escritas locais.

> O caminho do spark é setado pelo script de configuração

In [6]:
import os

lake_prefix = "temp-output"

if datalake_mode in ('leitura','shared_drive',):
    warehouse_dir = f"/content/datalake/{lake_prefix}/"

if datalake_mode == 'escrita':
    warehouse_dir = f"/content/datalake/"

spark_path = os.getenv('SPARK_HOME')
spark_path

'/content/spark-3.4.4-bin-hadoop3'

## Inclusão da pasta do repositório no python path

Procedimento permite que funções e classes presentes no repositório sejam utilizadas

In [7]:
import sys
sys.path.append('/content/sus-kpis-analysis')
sys.path

['/content',
 '/env/python',
 '/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/usr/local/lib/python3.11/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.11/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/sus-kpis-analysis']

## Importação de funções utilizadas pelo código

In [8]:
from sia.etls.lib.catalog_loader import DeltaLakeDatabaseFsCreator, load_entire_catalog_fs, load_entire_catalog_fs_v2
from sia.etls.lib.table_utilities import vacuum_tables_from_database, table_exists
from sia.etls.lib.fs_spark_session import create_fs_spark_session
from sia.etls.lib.bronze_files_utilities import get_pending_files_from_bronze
from sia.etls.lib.delta_table_creators import ParquetToDelta

## Cria Sessão Spark conectada ao Delta Lake presente no Google Storage

In [9]:
spark = create_fs_spark_session(
    warehouse_dir=warehouse_dir,
    spark_path=spark_path
)


## Refresh do catálogo para utilizar consultas

In [10]:
zone_names = ['monitor-rosa-bronze', 'monitor-rosa-silver', 'monitor-rosa-gold']
zone_names = ['monitor-rosa-silver', 'monitor-rosa-gold']
if datalake_mode in ('leitura', 'escrita'):
    zone_paths = [f'/content/datalake/{zone_name}/databases' for zone_name in zone_names]
else:
    zone_paths = [f'/content/drive/Shareddrives/{zone_name}/databases' for zone_name in zone_names]


# carrega catalogo de banco de dados, na zona bronze
database_filter = None #['cnes_bronze.db',]
table_filter = None #['cnes_bronze.sr',]

for databases_path in zone_paths:
    load_entire_catalog_fs_v2(
        spark_session = spark,
        databases_path = databases_path,
        use_db_folder_path=(datalake_mode == 'escrita'),
        database_filter=database_filter,
        table_filter=table_filter
    )

['cancer_data.db', 'ibge_silver.db', 'cancer_mama_silver.db']
Banco de dados cancer_data criado.
listando conteúdos do caminho /content/drive/Shareddrives/monitor-rosa-silver/databases e database cancer_data
prefix: /content/drive/Shareddrives/monitor-rosa-silver/databases/cancer_data.db/
table_list: ['aq_filtered', 'ar_filtered', 'dados_estados_mensal', 'dados_municipios_mensal', 'pacientes', 'procedimentos', 'procedimentos_e_pacientes', 'demografia_municipios', 'cadastro_municipios']
Tabela aq_filtered criada
Tabela aq_filtered criada com comando CREATE TABLE IF NOT EXISTS cancer_data.aq_filtered USING delta LOCATION '/content/drive/Shareddrives/monitor-rosa-silver/databases/cancer_data.db/aq_filtered'
Tabela ar_filtered criada
Tabela ar_filtered criada com comando CREATE TABLE IF NOT EXISTS cancer_data.ar_filtered USING delta LOCATION '/content/drive/Shareddrives/monitor-rosa-silver/databases/cancer_data.db/ar_filtered'
Tabela dados_estados_mensal criada
Tabela dados_estados_mensal 

# Exemplos de consultas

## Listagem de bancos e tabelas

In [11]:
databases = spark.sql(f"SHOW DATABASES;")
databases.show()

+------------------+
|         namespace|
+------------------+
|       cancer_data|
|       cancer_mama|
|cancer_mama_silver|
|           default|
|       ibge_silver|
+------------------+



In [12]:
for row in databases.collect():
    spark.sql(f"SHOW TABLES FROM {row['namespace']};").show(truncate=False)

+-----------+-------------------------+-----------+
|namespace  |tableName                |isTemporary|
+-----------+-------------------------+-----------+
|cancer_data|aq_filtered              |false      |
|cancer_data|ar_filtered              |false      |
|cancer_data|cadastro_municipios      |false      |
|cancer_data|dados_estados_mensal     |false      |
|cancer_data|dados_municipios_mensal  |false      |
|cancer_data|demografia_municipios    |false      |
|cancer_data|pacientes                |false      |
|cancer_data|procedimentos            |false      |
|cancer_data|procedimentos_e_pacientes|false      |
+-----------+-------------------------+-----------+

+-----------+-------------------------+-----------+
|namespace  |tableName                |isTemporary|
+-----------+-------------------------+-----------+
|cancer_mama|dados_estados_mensal     |false      |
|cancer_mama|dados_municipios_mensal  |false      |
|cancer_mama|pacientes                |false      |
|cancer_mam

In [13]:
dados_estad_mensal_df = spark.sql(f"""
    SELECT
        year(to_date(data, 'yyyyMM')) AS ano
        ,primeiro_estadiamento
        --SUM(custo) AS custo_estadiamento
        ,COUNT(DISTINCT(paciente)) AS numero_pacientes
        --,SUM(DISTINCT(obito)) AS obitos
        --,SUM(DISTINCT(indicacao_obito)) AS obito_futuro
        --,COUNT(1) AS numero_procedimentos
    FROM
        (SELECT * FROM cancer_mama.procedimentos_e_pacientes ORDER BY data)
    GROUP BY ano, primeiro_estadiamento
""")


In [14]:
df = dados_estad_mensal_df.toPandas()

In [15]:
# prompt: Usando o DataFrame df: stacked percentage of numero_pacientes by ano and estadiamento, using 0 in the bottom, and 1, 2, 3, 4 ordered to the top

import pandas as pd
import altair as alt

# Create a DataFrame (replace with your actual data)
# Assuming your DataFrame is named 'df'

# Convert 'primeiro_estadiamento' to numeric, handling errors
df['primeiro_estadiamento'] = pd.to_numeric(df['primeiro_estadiamento'], errors='coerce')

# Group by 'ano' and 'primeiro_estadiamento'
grouped_data = df.groupby(['ano', 'primeiro_estadiamento'])['numero_pacientes'].sum().reset_index()

# Calculate the total number of patients for each year
total_patients = grouped_data.groupby('ano')['numero_pacientes'].sum()

# Calculate the percentage for each group within each year
grouped_data['percentage'] = grouped_data.apply(lambda x: (x['numero_pacientes'] / total_patients[x['ano']]) * 100, axis=1)

# Create the stacked percentage chart using Altair
chart = alt.Chart(grouped_data).mark_bar().encode(
    x='ano:N',  # Categorical x-axis for years
    y=alt.Y('percentage:Q', stack='normalize', axis=alt.Axis(format='%')), # Stacked percentage with percentage formatting
    color=alt.Color('primeiro_estadiamento:N', sort='ascending'),  # Color by stage
    order=alt.Order('primeiro_estadiamento:N', sort='ascending')  # Order bars by stage
).properties(
    title='Stacked Percentage of Patients by Year and Stage'
)
chart


alt.Chart(...)

In [16]:
# prompt: # prompt: Usando o DataFrame df crie uma tabela com o percentual de numero_pacientes por ano e estadiamento, coloque os valores percentuais em colunas, seguindo a sequencia 0, 1, 2 , 3 e 4. Cada ano deve ser uma linha.

import pandas as pd
# Pivot the table to have stages as columns and years as rows
pivot_df = pd.pivot_table(grouped_data, values='percentage', index='ano', columns='primeiro_estadiamento', aggfunc='sum', fill_value=0)

# Rename columns for clarity (optional)
pivot_df = pivot_df.rename(columns={0: '0%', 1: '1%', 2: '2%', 3: '3%', 4: '4%'})

# Display the resulting table
pivot_df


primeiro_estadiamento,0%,1%,2%,3%,4%
ano,,,,,
2008,3.816270,19.432165,38.608962,27.402961,10.739643
2009,3.783668,19.806602,37.897909,28.969949,9.541872
2010,3.800870,20.184736,37.048134,30.150258,8.816001
2011,3.734570,20.765608,36.429540,30.771316,8.298965
2012,3.957398,21.278319,35.998380,30.697724,8.068179
2013,4.017623,21.591448,35.429096,31.020194,7.941639
2014,3.614945,21.704322,35.279707,31.274432,8.126593
2015,3.458813,21.376709,34.668993,32.079504,8.415982
2016,3.521466,21.234751,34.212331,32.366323,8.665129


In [20]:
spark.sql(f"""
    SELECT
        COUNT(DISTINCT(paciente)) AS numero_pacientes
    FROM
        cancer_mama.procedimentos_e_pacientes
    where data = '202501'

""").show()

+----------------+
|numero_pacientes|
+----------------+
|          143671|
+----------------+



In [19]:
spark.sql(f"""
    SELECT
        COUNT(DISTINCT(paciente)) AS numero_pacientes
    FROM
        cancer_mama.procedimentos_e_pacientes

""").show()

+------+--------+------------+------+-----+---------+--------------------------+------------------------+---------------------+------------------+-------------------+-------------------+------------------+----------------+---------------+
|  data|paciente|estadiamento| custo|obito|municipio|data_primeiro_estadiamento|data_ultimo_estadiamento|primeiro_estadiamento|maior_estadiamento|ultimo_estadiamento|        custo_total|primeiro_municipio|ultimo_municipio|indicacao_obito|
+------+--------+------------+------+-----+---------+--------------------------+------------------------+---------------------+------------------+-------------------+-------------------+------------------+----------------+---------------+
|202405|        |           1| 79.75|    0|   354890|                    202404|                  202502|                    3|                 4|                  4|3.812679215000043E7|            351930|          250130|              1|
|202412|        |           4|  34.1|    0| 

In [27]:
spark.sql(f"""
    SELECT
        sum(custo) AS custo_pacientes
    FROM
        cancer_mama.procedimentos_e_pacientes
        where data = '202501'

""").show()

+-------------------+
|    custo_pacientes|
+-------------------+
|7.746116285000196E7|
+-------------------+



In [25]:
spark.sql(f"""
    SELECT
        SUM(custo_estadiamento) AS custo_pacientes
    FROM
        cancer_mama.dados_estados_mensal
    where data = '202501'

""").show()

+-------------------+
|    custo_pacientes|
+-------------------+
|7.746116284999998E7|
+-------------------+



In [22]:
spark.sql(f"""
    SELECT
        SUM(numero_pacientes) AS numero_pacientes
    FROM
        cancer_mama.dados_estados_mensal
    where data = '202501'

""").show()

+----------------+
|numero_pacientes|
+----------------+
|          145963|
+----------------+



In [28]:
spark.sql(f"""
    SELECT
        SUM(numero_pacientes) AS numero_pacientes
    FROM
        cancer_mama.dados_municipios_mensal
    where data = '202501'

""").show()

+----------------+
|numero_pacientes|
+----------------+
|          145963|
+----------------+



In [23]:
spark.sql(f"""
    SELECT
        *
    FROM
        cancer_mama.dados_estados_mensal


""").show()

+--------+------+---------------------+------------------+----------------+-----------------+------+--------------+--------------------+-------------------+
|  estado|  data|primeiro_estadiamento|custo_estadiamento|numero_pacientes|numero_municipios|obitos|obitos_futuros|numero_procedimentos|numero_diagnosticos|
+--------+------+---------------------+------------------+----------------+-----------------+------+--------------+--------------------+-------------------+
|Rondônia|200801|                    1|1437.6999999999998|              13|                7|     0|             0|                  13|                 13|
|Rondônia|200801|                    2|17133.420000000002|              52|               19|     0|             2|                  53|                 52|
|Rondônia|200801|                    3|           50399.3|              70|               25|     0|             2|                  71|                 70|
|Rondônia|200801|                    4|          34021.97|

In [45]:
spark.sql(f"""
    select sum(num_mun), count(1) from

    (SELECT
        count(distinct municipio) num_mun, paciente


    FROM
        cancer_mama.procedimentos_e_pacientes
     where data = '202501'
     group by paciente)

     where num_mun > 1

""").show()

+------------+--------+
|sum(num_mun)|count(1)|
+------------+--------+
|        2409|     117|
+------------+--------+



In [44]:
143671 - 145963

-2292

In [46]:
2409 - 117

2292

In [47]:
spark.sql(f"""
    select * from

    (SELECT
        count(distinct municipio) num_mun, paciente


    FROM
        cancer_mama.procedimentos_e_pacientes
     where data = '202501'
     group by paciente)

     where num_mun > 1

""").show()

+-------+---------------+
|num_mun|       paciente|
+-------+---------------+
|   1792|               |
|    387|{{{{{{{{{{{{{{{|
|      2||{~{{{{|
|      2|}{||{{{{|
|      2|}{}{{{{|
|      2|}{|~~{{{|
|      2|{{{{|{|
|      2|{{{{}}}~{|
|      2|{{{{~~{~{|
|      2|{{{{~}{{~|
|      2|{{{{~{{|
|      2|{{{{~}||{}|
|      2|{{{{}|{{|
|      2|{{{{{|{|
|      2|{{{{|~{|
|      2|{{{{{}{~|
|      2|{{{{{}~|{|
|      2|{{{{}~}{|
|      2|{{}{{}~|
|      2|{{}{{}|
+-------+---------------+
only showing top 20 rows



In [49]:
spark.sql(f"""
    select * from

    (SELECT
        count(distinct municipio) num_mun, paciente


    FROM

  (SELECT
      c.*,
      p.data_primeiro_estadiamento,
      p.data_ultimo_estadiamento,
      p.primeiro_estadiamento,
      p.maior_estadiamento,
      p.ultimo_estadiamento,
      p.custo_total,
      p.primeiro_municipio,
      p.ultimo_municipio,
      p.indicacao_obito
  FROM cancer_mama.pacientes AS p
  LEFT JOIN cancer_mama.procedimentos AS c
  ON c.paciente = p.paciente)

     where data = '202501'
     group by paciente)

     where num_mun > 1

""").show()

+-------+---------------+
|num_mun|       paciente|
+-------+---------------+
|    387|{{{{{{{{{{{{{{{|
|   1792|               |
|      2|{{}~{~}}{||
|      2|{~{{|~~||
|      2|{{~{{|~{|
|      2|}{||{{{{|
|      2|{{||
|      2|{{}}}|{~|
|      2||{~{{{{|
|      2|{{{~}{}}}|
|      2|{}{{~}~||
|      2|{}{}~|
|      2|{~{~|{}|
|      2|{{}{{|}{|
|      2|{{{{}|
|      2|{{{{~}||{}|
|      2|{{{{}}}~{|
|      2|{{{{~}{{~|
|      2|{{}|{}|{||
|      2|{{}~||~|
+-------+---------------+
only showing top 20 rows

